In [43]:
import numpy as np
import pandas as pd
from pathlib import Path
import re

In [49]:
column_names = ["id", "agent", "exponential_decay", "step_decay", "sgdr", "constant"]

# Read the CSV file, skipping the first row and assigning column names
df = pd.read_csv('../data/data.csv', header=None, names=column_names, skiprows=1)

# re.match("\d+\.\d+)e([+-]\d+) ± \d+\.\d+)e([+-]\d+)", )
df

,id,agent,exponential_decay,step_decay,sgdr,constant
0,0,teacher,6.953e+00 ± 3.633e+00,6.917e+00 ± 3.656e+00,6.603e+00 ± 3.822e+00,6.934e+00 ± 3.648e+00
1,1,bc,6.894e+00 ± 3.723e+00,6.746e+00 ± 3.804e+00,6.473e+00 ± 3.952e+00,6.645e+00 ± 3.842e+00
2,2,td3_bc,6.371e+00 ± 3.940e+00,6.518e+00 ± 3.900e+00,6.609e+00 ± 3.837e+00,6.182e+00 ± 3.968e+00
3,3,cql,7.926e+00 ± 3.965e+00,6.973e+00 ± 3.893e+00,7.348e+00 ± 3.492e+00,9.322e+00 ± 3.943e+00
4,4,awac,6.838e+00 ± 3.739e+00,6.691e+00 ± 3.817e+00,6.632e+00 ± 3.833e+00,6.599e+00 ± 3.854e+00
5,5,edac,1.153e+01 ± 2.146e+00,6.781e+00 ± 3.791e+00,7.134e+00 ± 3.413e+00,9.857e+00 ± 3.880e+00
6,6,sac_n,6.490e+00 ± 3.858e+00,6.887e+00 ± 4.483e+00,1.023e+01 ± 3.132e+00,6.019e+00 ± 4.314e+00
7,7,lb_sac,7.891e+00 ± 4.431e+00,1.001e+01 ± 4.079e+00,1.023e+01 ± 3.716e+00,9.421e+00 ± 2.966e+00
8,8,iql,6.685e+00 ± 3.804e+00,6.700e+00 ± 3.795e+00,6.839e+00 ± 3.296e+00,6.706e+00 ± 3.798e+00


In [46]:
def extract_mean(value):
    mean_str = re.search(r'([\d\.e\+\-]+)\s±\s[\d\.e\+\-]+', value).group(1)
    return float(mean_str)

In [50]:
teacher_row = df[df["agent"] == "teacher"].iloc[0]

def format_value(row, column):
    mean = extract_mean(row[column])
    teacher_mean = extract_mean(teacher_row[column])
    value_str = row[column]
    
    if row.name == min_indices[column]:
        value_str = f"\\textbf{{{value_str}}}"
    if mean < teacher_mean:
        value_str = f"\\underline{{{value_str}}}"
    
    return value_str

In [51]:
min_indices = {}
for column in ["exponential_decay", "step_decay", "sgdr", "constant"]:
    df[f"{column}_mean"] = df[column].apply(extract_mean)
    min_indices[column] = df[f"{column}_mean"].idxmin()

highlighted_df = df.copy()

for column in ["exponential_decay", "step_decay", "sgdr", "constant"]:
    highlighted_df[column] = highlighted_df.apply(lambda row: format_value(row, column), axis=1)

# Drop the temporary mean columns from the DataFrame
highlighted_df.drop(columns=[f"{col}_mean" for col in ["exponential_decay", "step_decay", "sgdr", "constant"]], inplace=True)

# Convert the highlighted DataFrame to a LaTeX table
latex_table = highlighted_df.to_latex(index=False, escape=False)
print(latex_table)



\begin{tabular}{rlllll}
\toprule
 id &   agent &                          exponential_decay &                                 step_decay &                                       sgdr &                                   constant \\
\midrule
  0 & teacher &                      6.953e+00 ± 3.633e+00 &                      6.917e+00 ± 3.656e+00 &                      6.603e+00 ± 3.822e+00 &                      6.934e+00 ± 3.648e+00 \\
  1 &      bc &          \underline{6.894e+00 ± 3.723e+00} &          \underline{6.746e+00 ± 3.804e+00} & \underline{\textbf{6.473e+00 ± 3.952e+00}} &          \underline{6.645e+00 ± 3.842e+00} \\
  2 &  td3_bc & \underline{\textbf{6.371e+00 ± 3.940e+00}} & \underline{\textbf{6.518e+00 ± 3.900e+00}} &                      6.609e+00 ± 3.837e+00 &          \underline{6.182e+00 ± 3.968e+00} \\
  3 &     cql &                      7.926e+00 ± 3.965e+00 &                      6.973e+00 ± 3.893e+00 &                      7.348e+00 ± 3.492e+00 &                    

/tmp/ipykernel_5850/122864883.py:15: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = highlighted_df.to_latex(index=False, escape=False)
